In [ ]:
%run Lux_Hierarchy.ipynb  

In [ ]:
b_vals = [0, 0.1, 0.25, 0.5, 1, 2]

def strength_experiment(n = 400, l = 5, c = 5):
    
    global info_efficiency
    info_efficiency = 0.5
    
    global echo_chamber
    echo_chamber = 0
    
    results = []
    
    
    for v in b_vals:
        global b
        b = v
        m = Market()
        price_hist, count_hist, pf_hist = run_market(n)
        
        tmp = {}
        r = gsadf_test(price_hist)
        
        tmp["Strength"] = b
        tmp["GSADF"] = r
        tmp["VOLATILITY"] = statistics.stdev(price_hist)
        
        results.append(tmp)
       
    
    df = pd.DataFrame(results)
    fname = 'strength' + '-N' + str(n) + '-pset' + str(PARAMETER_SET) + '-echo' + str(echo_chamber) + '.csv'
    df.to_csv(fname, mode='a', index=False, header= not os.path.exists(fname))

    
def run_strength_experiments(n_experiments, n = 400):
    for i in range(n_experiments):
        clear_output(wait=False)
        print(str(i + 1) + '/' + str(n_experiments))
        strength_experiment(n)

In [ ]:
efficiency_vals = [0.1, 0.25, 0.5, 1, 2, 4]

def efficiency_experiment(n = 400):
    global echo_chamber
    echo_chamber = 0
    
    results = []
    for e in efficiency_vals:
        global info_efficiency
        info_efficiency = e
        
        price_hist, count_hist, pf_hist = run_market(n)
        
        tmp = {}
        tmp["EFFICIENCY"] = e
        tmp["GSADF"] = gsadf_test(price_hist)
        tmp["VOLATILITY"] = statistics.stdev(price_hist)
        tmp["FUNDAMENTAL_DEV"] = get_pf_deviation(price_hist, pf_hist)
        
        results.append(tmp)
        
    df = pd.DataFrame(results)
    fname = 'efficiency' + '-N' + str(n) + '-pset' + str(PARAMETER_SET) + '.csv'
    df.to_csv(fname, mode='a', index=False, header= not os.path.exists(fname))
    
    
def run_efficiency_experiments(n_experiments, n = 400):
    for i in range(n_experiments):
        clear_output(wait=False)
        print(str(i + 1) + '/' + str(n_experiments))
        efficiency_experiment(n)

In [ ]:
shape_vals = [(2, 500), (3, 23), (4, 8), (5, 5), (7, 3), (10, 2)]

def shape_experiment(n = 400):
    
    global info_efficiency
    info_efficiency = 0.5
    
    global echo_chamber
    echo_chamber = 0
    
    results = []
    for shape in shape_vals:
        global levels
        levels = shape[0]
    
        global children_per_agent
        children_per_agent = shape[1]
        
        price_hist, count_hist, pf_hist = run_market(n)
        
        tmp = {}
        tmp["SHAPE"] = shape
        tmp["GSADF"] = gsadf_test(price_hist)
        tmp["VOLATILITY"] = statistics.stdev(price_hist)
        tmp["FUNDAMENTAL_DEV"] = get_pf_deviation(price_hist, pf_hist)
        
        results.append(tmp)
        
        
    df = pd.DataFrame(results)
    fname = 'shape' + '-N' + str(n) + '-pset' + str(PARAMETER_SET) + '.csv'
    df.to_csv(fname, mode='a', index=False, header= not os.path.exists(fname))
    
    
def run_shape_experiments(n_experiments, n = 400):
    for i in range(n_experiments):
        clear_output(wait=False)
        print(str(i + 1) + '/' + str(n_experiments))
        shape_experiment(n)

In [ ]:
def stylised_experiment(n = 400):
    global info_efficiency
    info_efficiency = 0.5
    
    global echo_chamber
    echo_chamber = 0
    
    price_hist, count_hist, pf_hist = run_market(n)
    
    results = []
        
    tmp = {}
    tmp["Tails10"] = get_pareto_exponent(price_hist, 0.1)
    tmp["Tails5"] = get_pareto_exponent(price_hist, 0.05)
    tmp["Tails2.5"] = get_pareto_exponent(price_hist, 0.025)
    tmp["Kurt50"] = get_kurtosis(price_hist, 50)[0]
    tmp["Kurt10"] = get_kurtosis(price_hist, 10)[0]
    tmp["Kurt1"] = get_kurtosis(price_hist, 1)[0]
    
    
    autocorr_abs, autocorr_sq = calculate_autocorr(price_hist)
    x = np.asarray(range(1, len(autocorr_abs) + 1))
    y = np.asarray(list(autocorr_abs))
    parameters, _ = curve_fit(pow_law, x, y, maxfev=5000)
    
    tmp["Autocorr_Decay"] = abs(parameters[0])
    tmp["Abs_Autocorr_Lag10"] = autocorr_abs[10]
    tmp["Sq_Autocorr_Lag10"] = autocorr_sq[10]
    
    results.append(tmp)
        
    df = pd.DataFrame(results)
    fname = 'stylised'  + '-N' + str(n) + '-pset' + str(PARAMETER_SET)  + '-b' + str(mult) + 'a1' + '.csv'
    df.to_csv(fname, mode='a', index=False, header= not os.path.exists(fname))
    
    
def run_stylised_experiments(n_experiments, n = 400):
    for i in range(n_experiments):
        clear_output(wait=False)
        print(str(i + 1) + '/' + str(n_experiments))
        stylised_experiment(n)

In [ ]:
echo_vals = [0, 0.25, 0.5, 1, 2]

def echo_experiment(n = 400, symmetric = True):
    global info_efficiency
    info_efficiency = 0.5
    
    global symmetric_echo
    symmetric_echo = symmetric
    
    results = []
    
    for v in echo_vals:
        global echo_chamber
        echo_chamber = v
        
        price_hist, count_hist, pf_hist = run_market(n)
        
        tmp = {}
        tmp["ECHO"] = v
        tmp["GSADF"] = gsadf_test(price_hist)
        tmp["VOLATILITY"] = statistics.stdev(price_hist)
        tmp["FUNDAMENTAL_DEV"] = get_pf_deviation(price_hist, pf_hist)
        
        results.append(tmp)
        
    df = pd.DataFrame(results)
    
    if symmetric:
        fname = 'SymEcho' + '-N' + str(n) + '-pset' + str(PARAMETER_SET) + '-b' + str(b) + '.csv'
    else:
        fname = 'AsymEcho' + '-N' + str(n) + '-pset' + str(PARAMETER_SET) + '-b' + str(b) + '.csv'
    
    df.to_csv(fname, mode='a', index=False, header= not os.path.exists(fname))

def run_echo_experiments(n_experiments, n = 400):
    for i in range(n_experiments):
        clear_output(wait=False)
        print(str(i + 1) + '/' + str(n_experiments))
        echo_experiment(n)